##### Prerequisities 

Make sure you choose `PyTorch 1.6 Python 3.6 GPU Optimized` for **Kernel** above

In [ ]:
%%capture

!pip install transformers==4.6.1
!pip install datasets[s3]==1.6.2

Upgrade ipywidgets for `datasets` library and restart kernel

In [ ]:
%%capture

import IPython
!conda install -c conda-forge ipywidgets -y
IPython.Application.instance().kernel.do_shutdown(True)

#### Imports 

In [ ]:
from sagemaker import get_execution_role, Session
from datasets.filesystems import S3FileSystem
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer
import pandas as pd

In [ ]:
import transformers 
import sagemaker
import datasets
import logging
import torch
import sys

##### Setup Logger

In [ ]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

In [ ]:
logger.info(f'[Using SageMaker: {sagemaker.__version__}]')
logger.info(f'[Using Transformers: {transformers.__version__}]')
logger.info(f'[Using Datasets: {datasets.__version__}]')
logger.info(f'[Using Torch: {torch.__version__}]')

#### Essentials

In [ ]:
session = Session()
role = get_execution_role()
bucket = session.default_bucket()
s3 = S3FileSystem() 

In [ ]:
logger.info(f'Default Bucket = {bucket}')
logger.info(f'Sagemaker Role ARN ={role}')
logger.info(f'Sagemaker Session Region = {session.boto_region_name}')

#### Explore data set

In [ ]:
pd.set_option('max_colwidth', 400)

In [ ]:
train_df = pd.read_csv('./data/train.csv')
train_df.head(5)

In [ ]:
train_df.shape

In [ ]:
test_df = pd.read_csv('./data/test.csv')
test_df.shape

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

Tokenize

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', use_fast=True)

In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

In [ ]:
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=1000, num_proc=10)
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=1000, num_proc=10)

In [ ]:
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
train_dataset

In [ ]:
type(train_dataset)

Persist tokenized train and test sets from RAM to S3

In [ ]:
training_input_path = f's3://{bucket}/imdb/train'
train_dataset.save_to_disk(training_input_path,fs=s3)

test_input_path = f's3://{bucket}/imdb/test'
test_dataset.save_to_disk(test_input_path,fs=s3)